In [ ]:
import sys
import numpy as np
from matplotlib import pyplot as plt

# # uncomment this to add a path to another file you want to import, eg 'ModelSaver.py'
# sys.path.insert(0, "/home/luke/mymujoco/rl")

# robby TODO: write in the correct root to the folder containing our test files (pb4_tests_apr23)

# create modelsaver instance
from ModelSaver import ModelSaver
directory = "pb4_tests_apr23"
saver = ModelSaver(directory, root="/path/to/folder/containing/pb4_tests")

from grasp_test_data import GraspTestData
data_explorer = GraspTestData()

# what is the name of the test you would like to load
test_name = "pb4_EI1_S3"

# enter folder and load test data
saver.enter_folder(test_name)
try:
  testdata = saver.load(id=None, filenamestarts="test_data", suffix_numbering=True)
  print(data_explorer.get_test_string(data=testdata, detailed=True))
except TypeError as e:
  print("Loading test data failed, check that the test has actually saved some data")
  print("Error was:", e)

In [ ]:
load_test_infomation = False

if load_test_infomation:
  saver.enter_folder(test_name)
  try:
    testdata = saver.load(id=None, filenamestarts="test_data", suffix_numbering=True)
    print(data_explorer.get_test_string(data=testdata, detailed=True))
  except TypeError as e:
    print("Loading test data failed, check that the test has actually saved some data")
    print("Error was:", e)

In [ ]:
load_test_pictures = True

# what file? Choose X where 'trial_image_batch_X.pbz2'
picture_file_id = 40

# extract the image data
imagedata = saver.load(id=picture_file_id, filenamestarts="trial_image_batch")
image_list = imagedata.trials[0].images # always use trials[0]
print("There are", len(image_list), "images")

In [ ]:
# how many images would we like to plot
rows = 2
cols = 2

num_images = rows * cols
if num_images > len(image_list): num_images = len(image_list)

fig, axs = plt.subplots(rows, cols, sharex=True, sharey=True)
fig2, axs2 = plt.subplots(rows, cols, sharex=True, sharey=True)

# saturate any values above this (ignore outliers)
depth_max = 5000

# find minimum of minima & maximum of depth data
dmin = 1e10
dmax = -1e10
for i in range(len(image_list)):
  this_min = np.min(image_list[i].depth[image_list[i].depth < depth_max])
  this_max = np.max(image_list[i].depth[image_list[i].depth < depth_max])
  if this_min < dmin: dmin = this_min
  if this_max > dmax: dmax = this_max

for j in range(num_images):
  ix = j // cols
  iy = j % cols
  im1 = axs[ix][iy].imshow(image_list[j].rgb, aspect="auto")
  im2 = axs2[ix][iy].imshow(image_list[j].depth, vmin=dmin, vmax=dmax, aspect='auto', cmap="viridis")

fig2.subplots_adjust(right=0.8)
cbar_ax = fig2.add_axes([0.85, 0.15, 0.05, 0.7])
fig2.colorbar(im2, cax=cbar_ax)

fig.suptitle("RGB images")
fig2.suptitle("Depth images")

fig.set_size_inches((cols * 3, rows * 2))
fig2.set_size_inches((cols * 3, rows * 2))

plt.show()